In [164]:
# -*- coding:utf-8 -*-
import tushare as ts
import matplotlib.pyplot as plt
#import matplotlib.finance as mpf
import mpl_finance as mpf
import matplotlib.ticker as ticker

import datetime
import numpy as np
import pandas as pd
import time

from pyecharts import Overlap
from pyecharts import Kline
from pyecharts import Line
from pyecharts import Bar
from pyecharts import Grid

from pandas import DataFrame, Series


In [165]:
def n_days_ago(n):
    today=datetime.date.today()
    ndays_ago=today-datetime.timedelta(n)
    return str(ndays_ago)

def hhv(s, n):
    return Series.rolling(s, n).max()

def llv(s, n):
    return Series.rolling(s, n).min()

    
#ichimoku
def ichimoku(s, n1=9, n2=26, n3=52):

    #average of N-day high and N-day low
    conv = (hhv(s, n1) + llv(s, n1)) / 2

    #mid point of the latest 26 days
    base = (hhv(s, n2) + llv(s, n2)) / 2
    
    #mid-point between the first 2 lines,  and plot 26 periods ahead
    spana = (conv + base) / 2
    
    #mid-point between the 52-period low and  52-period high, and plot 26 periods
    spanb = (hhv(s, n3) + llv(s, n3)) / 2
    
    k = s

    #Lspan is closing price, and plot 26 periods in the past
    return DataFrame(dict(k=k,conv=conv, base=base, spana=spana.shift(n2),
                          spanb=spanb.shift(n2), lspan=s.shift(-n2)))

In [166]:
start_date = '2018-07-01'
now = int(time.time())
timeArray = time.localtime(now)
Time = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
end_date =  Time 
ndays = 180
stock_selected = '600487'
MAX_SMA=100
start_date = n_days_ago(ndays + MAX_SMA)
ktype = '240'

In [167]:
df = ts.get_k_data(stock_selected,start_date,end_date,ktype = '60')
v_kline = np.array(df[['open','close','low','high']][MAX_SMA:])
date_kline = []
for i,j in enumerate(np.array(df[['date']][MAX_SMA:])):
    #print(j)
    temp_str = str(j[0]).replace('-0','/')
    date_kline.append(temp_str.replace('-','/'))

In [168]:
SMA5 = np.array(pd.Series.rolling(df[['close']],5).mean()[MAX_SMA:])
SMA100 = np.array(pd.Series.rolling(df[['close']],100).mean()[MAX_SMA:])
SMA55 = np.array(pd.Series.rolling(df[['close']],55).mean()[MAX_SMA:])
v_volume = np.array(df['volume'][MAX_SMA:])

In [169]:
kline = Kline("K 线图示例")
kline.add("日K",
          date_kline,
          v_kline,
          mark_point=["max","min"],
          mark_line=["max"],
          is_datazoom_show=True
         )

sma_line = Line(width=10)

volume_bar = Bar()
volume_bar.add("",date_kline, v_volume)

overlap = Overlap()
sma_line.add("SMA5", date_kline,SMA5,is_symbol_show=False)
sma_line.add("SMA55",date_kline,SMA55,is_symbol_show=False)
sma_line.add("SMA100",date_kline,SMA100,is_symbol_show=False)

#ichimoku fill between
#plt.fill_between(df.index, df['spana'], df['spanb'], where=df['spanb'] >= df['spana'], facecolor='red', interpolate=True)
#plt.fill_between(df.index, df['spana'], df['spanb'], where=df['spanb'] <= df['spana'], facecolor='green', interpolate=True)
df_ich =  ichimoku(df['close'])
sma_line.add("base", date_kline,np.array(df_ich['base'])[MAX_SMA:], is_symbol_show=False, is_more_utils=True)

sma_line.add("conv", date_kline,np.array(df_ich['conv'])[MAX_SMA:],is_symbol_show=False,is_more_utils=True)

sma_line.add("spana", date_kline,np.array(df_ich['spana'])[MAX_SMA:],is_symbol_show=False,is_more_utils=True,
             is_fill=True, line_opacity=0.2,   area_opacity=0.7)
sma_line.add("spanb", date_kline,np.array(df_ich['spanb'])[MAX_SMA:],is_symbol_show=False,is_more_utils=True,
            is_fill=True, line_opacity=0.2,   area_opacity=0.3)
#l = Line() 
#l.add('WMA20', date, wma20.round(2), line_color='#8000ff', is_symbol_show=False, ) 
for i in range(5):
    sma_line._option['series'][i]['lineStyle']['normal']['width']=4

overlap.add(kline)
overlap.add(sma_line)

grid = Grid(height=600,width=1000)
grid.add(overlap, grid_bottom="55%")
grid.add(volume_bar, grid_top="55%")
#grid.render()

In [170]:
#sma_line.print_echarts_options()
#kline._option['series'][0]['itemStyle']['normal']['borderColor'] = NULL
sma_line._option['series'][1]['lineStyle']['normal']={
                    "opacity": 1,
                    "type": "solid",
                    "curveness": 0,
                    "width": 4
}

In [171]:
kline.print_echarts_options()

function kline_tooltip_formatter(params) {
    var text;
    text = ((((((((((((params[0].seriesName + "<br/>") + "- open:") + params[0].data[1]) + "<br/>") + "- close:") + params[0].data[2]) + "<br/>") + "- lowest:") + params[0].data[3]) + "<br/>") + "- highest:") + params[0].data[4]);
    return text;
}

{
    "title": [
        {
            "text": "K \u7ebf\u56fe\u793a\u4f8b",
            "left": "auto",
            "top": "auto",
            "textStyle": {
                "fontSize": 18
            },
            "subtextStyle": {
                "fontSize": 12
            }
        }
    ],
    "toolbox": {
        "show": true,
        "orient": "vertical",
        "left": "95%",
        "top": "center",
        "feature": {
            "saveAsImage": {
                "show": true,
                "title": "save as image"
            },
            "restore": {
                "show": true,
                "title": "restore"
            },
            "dataView": {
           